In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/riiid-test-answer-prediction/example_test.csv
/kaggle/input/riiid-test-answer-prediction/train.csv
/kaggle/input/riiid-test-answer-prediction/lectures.csv
/kaggle/input/riiid-test-answer-prediction/questions.csv
/kaggle/input/riiid-test-answer-prediction/example_sample_submission.csv
/kaggle/input/riiid-test-answer-prediction/riiideducation/competition.cpython-37m-x86_64-linux-gnu.so
/kaggle/input/riiid-test-answer-prediction/riiideducation/__init__.py


In [2]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.metrics import roc_auc_score
import tensorflow as tf
from sklearn.model_selection import train_test_split
import gc
import riiideducation
env = riiideducation.make_env()



In [3]:
dir_path = '/kaggle/input/riiid-test-answer-prediction/'
file_train = 'train.csv'
file_questions = 'questions.csv'

# nrows =  100 * 10000
nrows = None

# 压缩内存
max_num = 126

train = pd.read_csv(
                    dir_path + file_train, 
                    nrows=nrows, 
                    usecols=['row_id', 'timestamp', 'user_id', 'content_id', 
                             'content_type_id', 'task_container_id', 'answered_correctly',
                            'prior_question_elapsed_time','prior_question_had_explanation'],
                    dtype={
                            'row_id': 'int64',
                            'timestamp': 'int64',
                            'user_id': 'int32',
                            'content_id': 'int16',
                            'content_type_id': 'int8',
                            'task_container_id': 'int8',
#                           'user_answer': 'int8',
                            'answered_correctly': 'int8',
                            'prior_question_elapsed_time': 'float32',
                            'prior_question_had_explanation': 'str'
                        }
                   )

questions = pd.read_csv(
                        dir_path + file_questions, 
                        nrows=nrows,
                        usecols=['question_id','bundle_id','part'], 
                        dtype={
                           'question_id': 'int16',
                           'bundle_id': 'int16',
#                           'correct_answer':'int8',
                           'part': 'int8',
#                            'tags': 'str',#序列的加入没有带来太大提升
                       }
                    )


train['prior_question_had_explanation'] = train['prior_question_had_explanation'].map({'True':1,'False':0}).fillna(-1).astype(np.int8)

train = train[train['content_type_id']==0]

gc.collect()

train = train.groupby(['user_id']).tail(max_num)

train = pd.merge(
        left=train,
        right=questions,
        how='left',
        left_on='content_id',
        right_on='question_id'
        )

train = train.fillna(0)
del train["question_id"]
print(gc.collect())

0


# 计算user和题目的题目答对与否的特征(在测试集中需要实时更新)

In [4]:
user_agg = train.groupby('user_id')["answered_correctly"].agg(['sum', 'count'])
content_agg = train.groupby('content_id')["answered_correctly"].agg(['sum', 'count'])

# 计算题目的难度和已回答该题目的数量

In [5]:
train['content_count'] = train['content_id'].map(content_agg['count']).astype('int32')
train["content_count"]=train["content_count"].fillna(1)
train['question_difficulty'] = train['content_id'].map(content_agg['sum'] / content_agg['count'])

# 计算用户实时的答题正确率

In [6]:
train['lag'] = train.groupby('user_id')['answered_correctly'].shift()#标签沿着列的方向下移一位
cum = train.groupby('user_id')['lag'].agg(['cumsum', 'cumcount'])
train['user_correctness'] = cum['cumsum'] / cum['cumcount']#计算实时准确率
train.drop(columns=['lag'], inplace=True)
train["user_correctness"]=train["user_correctness"].fillna(0.7)

In [7]:
train.tail()

,row_id,timestamp,user_id,content_id,content_type_id,task_container_id,answered_correctly,prior_question_elapsed_time,prior_question_had_explanation,bundle_id,part,content_count,question_difficulty,user_correctness
25105220,101230327,428564420,2147482888,3586,0,22,1,18000.0,1,3586,5,981,0.745158,0.500000
25105221,101230328,428585000,2147482888,6341,0,23,1,14000.0,1,6341,5,2148,0.514898,0.521739
25105222,101230329,428613475,2147482888,4212,0,24,1,14000.0,1,4212,5,8422,0.595702,0.541667
25105223,101230330,428649406,2147482888,6343,0,25,0,22000.0,1,6343,5,1196,0.627926,0.560000
25105224,101230331,428692118,2147482888,7995,0,26,1,29000.0,1,7995,5,1578,0.636882,0.538462


In [8]:
#处理类别特征
class cat_deal:
    def __init__(self):
        self.max_len = 0
        self.dict_map = {}
    
    def fit(self, cat_list):
        index = 1 
        for cat_i in cat_list:
            if cat_i not in self.dict_map:
                self.dict_map[cat_i] = index
                index += 1
        self.max_len = index + 1
        
    def transform(self, cat_list):
        cat_transform_list = []
        for cat_i in cat_list:
            if cat_i in self.dict_map:
                cat_transform_list.append(self.dict_map[cat_i])
            else:
                cat_transform_list.append(0)
        return cat_transform_list
#处理浮点特征
class float_deal:
    def __init__(self):
        self.max = 0
        self.min = 0
        self.max_min = 0 
        
    def fit(self, float_list):
        for float_i in float_list:
            if float_i < self.min:
                self.min = float_i
            if float_i > self.max:
                self.max = float_i
        self.max_min = self.max - self.min
        
    def transform(self, float_list):
        float_transform_list = []
        for float_i in float_list:
            if float_i < self.min:
                float_transform_list.append(0)
            elif float_i > self.max:
                float_transform_list.append(1)
            else:
                float_transform_list.append(float_i/self.max_min)
        return float_transform_list
    
#处理序列特征
class cat_seq_deal:
    def __init__(self,sep_len=6,sep=' '):
        self.sep_len=sep_len
        self.sep=sep
        self.max_len=0
        self.dict_map={}
        
    def fit(self,cat_seq_list):
        index=1
        for cat_seq_i in cat_seq_list:
            cat_seq_i =cat_seq_i.split(self.sep)
            for cat_i in cat_seq_i:
                if cat_i not in self.dict_map:
                    self.dict_map[cat_i]=index
                    index+=1
        self.max_len=index+1
    
    def transform(self,cat_seq_list):
        cat_transform_list=[]
        for cat_seq_i in cat_seq_list:
            cat_seq_i =cat_seq_i.split(self.sep)
            len_cat_seq_i=len(cat_seq_i)
            if len_cat_seq_i>=self.sep_len:
                cat_seq_i=cat_seq_i[:self.sep_len]
            else:
                cat_seq_i=cat_seq_i+[0]*(self.sep_len-len_cat_seq_i)
            cat_seq_n=[]
            for cat_i in cat_seq_i:
                if cat_i in self.dict_map:
                    cat_seq_n.append(self.dict_map[cat_i])
                else:
                    cat_seq_n.append(0)
            cat_transform_list.append(cat_seq_n)
        return cat_transform_list  


In [9]:
dict_cat_class = {}
for columns in ['user_id','content_id',\
                'task_container_id','prior_question_had_explanation',\
                'bundle_id','part']:
    dict_cat_class[columns] = cat_deal()
    dict_cat_class[columns].fit(train[columns])

    train[columns] = dict_cat_class[columns].transform(train[columns])
    print(columns)


dict_float_class = {}
for columns in ['timestamp','prior_question_elapsed_time','content_count',\
                'question_difficulty','user_correctness']:
    dict_float_class[columns] = float_deal()
    dict_float_class[columns].fit(train[columns])
    
    train[columns] = dict_float_class[columns].transform(train[columns])
    print(columns)
    
# dict_cat_seq_class={}
# for columns in ['tags']:
#     dict_cat_seq_class[columns]=cat_seq_deal(sep_len=6,sep=' ')
#     dict_cat_seq_class[columns].fit(train[columns])
    
#     train[columns]=dict_cat_seq_class[columns].transform(train[columns])
#     print('tags')

                    

user_id
content_id
task_container_id
prior_question_had_explanation
bundle_id
part
timestamp
prior_question_elapsed_time
content_count
question_difficulty
user_correctness


In [10]:
train.tail()

,row_id,timestamp,user_id,content_id,content_type_id,task_container_id,answered_correctly,prior_question_elapsed_time,prior_question_had_explanation,bundle_id,part,content_count,question_difficulty,user_correctness
25105220,101230327,0.004902,393656,6749,0,23,1,0.060000,3,5480,1,0.007075,0.745158,0.500000
25105221,101230328,0.004902,393656,5999,0,24,1,0.046667,3,4896,1,0.015491,0.514898,0.521739
25105222,101230329,0.004903,393656,974,0,25,1,0.046667,3,806,1,0.060737,0.595702,0.541667
25105223,101230330,0.004903,393656,11068,0,26,0,0.073333,3,8341,1,0.008625,0.627926,0.560000
25105224,101230331,0.004903,393656,8139,0,27,1,0.096667,3,6432,1,0.011380,0.636882,0.538462


In [11]:
def squeeze(embedding):
    embedding = tf.squeeze(embedding,axis=1)
    return embedding
def concat(embedding_list):
    embedding = tf.concat(embedding_list, axis=1)
    return embedding
def multiply(multi_x_y):
    multi_x = multi_x_y[0]
    multi_y = multi_x_y[1]
    multi_x_y = tf.multiply(multi_x, multi_y)
    return multi_x_y
def add(add_x_y):
    multi_x=multi_x_y[0]
    multi_y=multi_x_y[1]
    multi_x_y=tf.add(multi_x,multi_x)
    return multi_x_y
#浮点数据分桶函数，处理后可当做类别特征使用
def cast(input_float,bin=200):
    float_2_cat=tf.cast(input_float/1.02*bin,dtype=tf.int64)
    return float_2_cat

In [12]:

#deepnFFm
# 浮点数据输入
input_timestamp = tf.keras.Input(shape=(1,))
input_prior_question_elapsed_time = tf.keras.Input(shape=(1,))
input_content_count = tf.keras.Input(shape=(1,))
input_question_difficulty = tf.keras.Input(shape=(1,))
input_user_correctness = tf.keras.Input(shape=(1,))

# 类别数据输入
input_user = tf.keras.Input(shape=(1,))
#可对用户进行类似item dropout的操作


input_content = tf.keras.Input(shape=(1,))
input_task_container = tf.keras.Input(shape=(1,))
input_prior_question_had_explanation = tf.keras.Input(shape=(1,))
input_bundle = tf.keras.Input(shape=(1,))
input_part = tf.keras.Input(shape=(1,))

#序列特征
# input_tags=tf.keras.Input(shape=(6,))
# embedding_tags_seq=tf.keras.layers.Embedding(dict_cat_seq_class['tags'].max_len,
#                                            64,input_length=1)(input_tags)
# embedding_tags_seq=tf.reduce_mean(embedding_tags_seq,axis=1)
# embedding_tags_seq=tf.keras.layers.Dense(64,activation=tf.nn.relu)(embedding_tags_seq)

# 所有输入
inputs = [input_question_difficulty,input_user_correctness,input_content_count,\
         input_timestamp,input_prior_question_elapsed_time,\
         input_user,input_content,\
         input_task_container,input_prior_question_had_explanation,\
         input_bundle,input_part
#           ,input_tags
         ]

# Wide部分
# 类别特征embeeding转换 
embedding_user_wide = tf.keras.layers.Embedding(dict_cat_class['user_id'].max_len,
                                           1, input_length=1)(input_user)
embedding_user_wide = tf.keras.layers.Lambda(squeeze)(embedding_user_wide)

embedding_content_wide = tf.keras.layers.Embedding(dict_cat_class['content_id'].max_len,
                                              1, input_length=1)(input_content)
embedding_content_wide = tf.keras.layers.Lambda(squeeze)(embedding_content_wide)

embedding_task_container_wide = tf.keras.layers.Embedding(dict_cat_class['task_container_id'].max_len,
                                                     1, input_length=1)(input_task_container)
embedding_task_container_wide = tf.keras.layers.Lambda(squeeze)(embedding_task_container_wide)

embedding_prior_question_had_explanation_wide = tf.keras.layers.Embedding(dict_cat_class['prior_question_had_explanation'].max_len, 
                                                                     1, input_length=1)(input_prior_question_had_explanation)
embedding_prior_question_had_explanation_wide = tf.keras.layers.Lambda(squeeze)(embedding_prior_question_had_explanation_wide)

embedding_bundle_wide = tf.keras.layers.Embedding(dict_cat_class['bundle_id'].max_len,
                                             1, input_length=1)(input_bundle)
embedding_bundle_wide = tf.keras.layers.Lambda(squeeze)(embedding_bundle_wide)

embedding_part_wide = tf.keras.layers.Embedding(dict_cat_class['part'].max_len,
                                           1, input_length=1)(input_part)
embedding_part_wide = tf.keras.layers.Lambda(squeeze)(embedding_part_wide)

# 连续值分桶,变成类别特征
input_timestamp_cat = tf.keras.layers.Lambda(cast)(input_timestamp)
input_prior_question_elapsed_time_cat = tf.keras.layers.Lambda(cast)(input_prior_question_elapsed_time)


embedding_timestamp_cat_wide = tf.keras.layers.Embedding(200,
                                           1, input_length=1)(input_timestamp_cat)
embedding_timestamp_cat_wide = tf.keras.layers.Lambda(squeeze)(embedding_timestamp_cat_wide)

embedding_prior_question_elapsed_time_cat_wide = tf.keras.layers.Embedding(200,
                                              1, input_length=1)(input_prior_question_elapsed_time_cat)
embedding_prior_question_elapsed_time_cat_wide = tf.keras.layers.Lambda(squeeze)(embedding_prior_question_elapsed_time_cat_wide)
# 合并类别特征对应的embeeding特征和浮点特征
embedding_all = [input_question_difficulty,input_user_correctness,input_content_count,\
                embedding_timestamp_cat_wide,embedding_prior_question_elapsed_time_cat_wide,\
                embedding_user_wide, embedding_content_wide, embedding_task_container_wide,\
                embedding_prior_question_had_explanation_wide, embedding_bundle_wide, embedding_part_wide]

wide_all = embedding_all
# wide layer（线性回归）
wide_layer = tf.keras.layers.Lambda(concat)(wide_all) 
wide_layer = tf.keras.layers.Dense(32,activation=tf.nn.relu)(wide_layer)



# FM和Deep部分
#连续特征
# embedding float
# embedding_timestamp_deep = tf.keras.layers.Dense(64, activation=tf.nn.relu)(input_timestamp)
# embedding_prior_question_elapsed_time_deep = tf.keras.layers.Dense(64, activation=tf.nn.relu)(input_prior_question_elapsed_time)
embedding_question_difficulty_deep = tf.keras.layers.Dense(64, activation=tf.nn.relu)(input_question_difficulty)
embedding_user_correctness_deep = tf.keras.layers.Dense(64, activation=tf.nn.relu)(input_user_correctness)
embedding_content_count_deep = tf.keras.layers.Dense(64, activation=tf.nn.relu)(input_content_count)

#分桶后的类别数据
embedding_timestamp_deep = tf.keras.layers.Embedding(200,
                                           64, input_length=1)(input_timestamp_cat)
embedding_timestamp_deep = tf.keras.layers.Lambda(squeeze)(embedding_timestamp_deep)

embedding_prior_question_elapsed_time_deep = tf.keras.layers.Embedding(200,
                                           64, input_length=1)(input_prior_question_elapsed_time_cat)
embedding_prior_question_elapsed_time_deep = tf.keras.layers.Lambda(squeeze)(embedding_prior_question_elapsed_time_deep)

# 类别特征embeeding转换 
embedding_user_deep = tf.keras.layers.Embedding(dict_cat_class['user_id'].max_len,
                                           64, input_length=1)(input_user)
embedding_user_deep = tf.keras.layers.Lambda(squeeze)(embedding_user_deep)

embedding_content_deep = tf.keras.layers.Embedding(dict_cat_class['content_id'].max_len,
                                              64, input_length=1)(input_content)
embedding_content_deep = tf.keras.layers.Lambda(squeeze)(embedding_content_deep)

embedding_task_container_deep = tf.keras.layers.Embedding(dict_cat_class['task_container_id'].max_len,
                                                     64, input_length=1)(input_task_container)
embedding_task_container_deep = tf.keras.layers.Lambda(squeeze)(embedding_task_container_deep)

embedding_prior_question_had_explanation_deep = tf.keras.layers.Embedding(dict_cat_class['prior_question_had_explanation'].max_len, 
                                                                     64, input_length=1)(input_prior_question_had_explanation)
embedding_prior_question_had_explanation_deep = tf.keras.layers.Lambda(squeeze)(embedding_prior_question_had_explanation_deep)

embedding_bundle_deep = tf.keras.layers.Embedding(dict_cat_class['bundle_id'].max_len,
                                             64, input_length=1)(input_bundle)
embedding_bundle_deep = tf.keras.layers.Lambda(squeeze)(embedding_bundle_deep)

embedding_part_deep = tf.keras.layers.Embedding(dict_cat_class['part'].max_len,
                                           64, input_length=1)(input_part)
embedding_part_deep = tf.keras.layers.Lambda(squeeze)(embedding_part_deep)

# 合并类别特征对应的embeeding特征和浮点特征
embedding_all = [embedding_question_difficulty_deep,embedding_user_correctness_deep,embedding_content_count_deep,embedding_timestamp_deep,embedding_prior_question_elapsed_time_deep,\
                embedding_user_deep, embedding_content_deep, embedding_task_container_deep,\
                embedding_prior_question_had_explanation_deep, embedding_bundle_deep, embedding_part_deep]



# FM部分(embedding+特征交叉对应元素相乘)
fm_all = embedding_all 
fm1, fm2 = [], []
for i, embedding_i in enumerate(embedding_all):
    for j, embedding_j in enumerate(embedding_all):
        if i > j:
            fm1.append(embedding_i), fm2.append(embedding_j)
fm1_layer = tf.keras.layers.Lambda(concat)(fm1)
fm2_layer = tf.keras.layers.Lambda(concat)(fm2)     

fm_layer = tf.keras.layers.Lambda(multiply)([fm1_layer,fm2_layer])
fm_layer=tf.nn.dropout(fm_layer,0.9)
fm_layer=tf.keras.layers.Dense(64, activation=tf.nn.relu)(fm_layer)

# Deep部分(embedding+DNN)
deep_all = embedding_all 
deep_layer = tf.keras.layers.Lambda(concat)(deep_all) 
#FC 
for unit_i in [16,32,16]:#[16,32,16]
    deep_layer = tf.keras.layers.Dense(unit_i, activation=tf.nn.relu)(deep_layer)
    #防止过拟合

    
# logit输出
deep_wide_fm_layer = tf.keras.layers.Lambda(concat)([deep_layer,wide_layer,fm_layer]) 
# deep_wide_fm_layer=tf.keras.layers.Dense(32, activation=tf.nn.relu)(deep_wide_fm_layer)
logit = tf.keras.layers.Dense(1, activation=tf.nn.sigmoid)(deep_wide_fm_layer)
#效果不好
# logit = tf.expand_dims(tf.nn.sigmoid(tf.reduce_sum(deep_wide_fm_layer,axis=1)),axis=1)

# 模型输入和输出
model = tf.keras.models.Model(inputs=inputs, outputs=logit)

In [13]:
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 1)]          0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 1)]          0                                            
__________________________________________________________________________________________________
lambda_6 (Lambda)               (None, 1)            0           input_1[0][0]                    
__________________________________________________________________________________________________
lambda_7 (Lambda)               (None, 1)            0           input_2[0][0]                    
_______________________________________________________________________________________

# 编译模型 设置参数

In [14]:

model.compile(loss='binary_crossentropy',
              optimizer= tf.keras.optimizers.Adam(lr=0.003),
              metrics=['binary_crossentropy',tf.keras.metrics.AUC()])

plateau = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_auc',
                            verbose=1,
                            mode='max',
                            factor=0.1,
                            patience=3)

early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_auc',
                               verbose=1,
                               mode='max',
                               patience=2)

# 保存
checkpoint = tf.keras.callbacks.ModelCheckpoint(f'fold.h5',
                             monitor='val_auc',
                             verbose=1,
                             mode='max',
                             save_best_only=True)


In [15]:

# 训练 验证
valid = pd.DataFrame()
#20 为了训练验证更稳定
for i in range(6):
    
    # 获取训练标签数据
    last_records = train.drop_duplicates('user_id', keep='last')
    
    # 获取训练标签以前的数据
    map__last_records__user_row = dict(zip(last_records['user_id'],last_records['row_id']))
    train['filter_row'] = train['user_id'].map(map__last_records__user_row)
    train = train[train['row_id']<train['filter_row']]

    # 特征加入训练集
    valid = valid.append(last_records)
    print(len(valid))


features_columns = ['question_difficulty','user_correctness','content_count',\
                    'timestamp','prior_question_elapsed_time',\
                    'user_id','content_id',\
                    'task_container_id','prior_question_had_explanation',\
                    'bundle_id','part'
#                     ,'tags'
                   ]

X_valid, y_valid = [np.array(valid[columns].values.tolist()) for columns in features_columns], valid['answered_correctly'].values
del valid
print("del")
X_train, y_train = [np.array(train[columns].values.tolist()) for columns in features_columns], train['answered_correctly'].values
del train
print("del")

393656
787225
1180734
1574202
1967602
2360984
del
del


In [16]:
#开始训练
model.fit(X_train, y_train,
          epochs=15,
          batch_size=512 *100* 2,
          verbose=1,
          shuffle=True,
          validation_data=(X_valid, y_valid),
          callbacks=[plateau, early_stopping, checkpoint])

y_valid_proba = model.predict(X_valid, verbose=0, batch_size=512)
auc = roc_auc_score(y_valid, y_valid_proba)
print(auc)


Epoch 1/15
223/223 [==============================] - ETA: 0s - loss: 0.5816 - binary_crossentropy: 0.5816 - auc: 0.7387
Epoch 00001: val_auc improved from -inf to 0.74218, saving model to fold.h5
223/223 [==============================] - 168s 752ms/step - loss: 0.5816 - binary_crossentropy: 0.5816 - auc: 0.7387 - val_loss: 0.5961 - val_binary_crossentropy: 0.5961 - val_auc: 0.7422
Epoch 2/15
223/223 [==============================] - ETA: 0s - loss: 0.5559 - binary_crossentropy: 0.5559 - auc: 0.7697
Epoch 00002: val_auc improved from 0.74218 to 0.74655, saving model to fold.h5
223/223 [==============================] - 167s 751ms/step - loss: 0.5559 - binary_crossentropy: 0.5559 - auc: 0.7697 - val_loss: 0.5946 - val_binary_crossentropy: 0.5946 - val_auc: 0.7466
Epoch 3/15
223/223 [==============================] - ETA: 0s - loss: 0.5456 - binary_crossentropy: 0.5456 - auc: 0.7798
Epoch 00003: val_auc did not improve from 0.74655
223/223 [==============================] - 166s 743ms/

In [17]:
model=tf.keras.models.load_model('fold.h5')
iter_test = env.iter_test()
prior_test_df = None
for (test_df, sample_prediction_df) in iter_test:
    #用预测完后的数据，更新训练样本中的用户题目特征
    if prior_test_df is not None:
        prior_test_df["answered_correctly"] = eval(test_df['prior_group_answers_correct'].iloc[0])
        prior_test_df = prior_test_df[prior_test_df["answered_correctly"] != -1].reset_index(drop=True)
        
        user_ids = prior_test_df['user_id'].values
        content_ids = prior_test_df['content_id'].values
        targets = prior_test_df["answered_correctly"].values
        
        for user_id, content_id, answered_correctly in zip(user_ids, content_ids, targets):
            if user_id in user_agg.index:
                user_agg.loc[user_id, 'sum'] += answered_correctly
                user_agg.loc[user_id, 'count'] += 1
            else:
                user_agg.loc[user_id] = [answered_correctly, 1]
            
            if content_id in content_agg.index:
                content_agg.loc[content_id, 'sum'] += answered_correctly
                content_agg.loc[content_id, 'count'] += 1
            else:
                content_agg.loc[content_id] = [answered_correctly, 1]

    prior_test_df = test_df.copy()
    
    test_df['prior_question_had_explanation'] = test_df['prior_question_had_explanation'].map({'True':1,'False':0}).fillna(-1).astype(np.int8)
    test_df = pd.merge(
        left=test_df,
        right=questions,
        how='left',
        left_on='content_id',
        right_on='question_id'
        )
    test_df['user_correctness'] = test_df['user_id'].map(user_agg['sum'] / user_agg['count'])
    
    test_df['content_count'] = test_df['content_id'].map(content_agg['count']).fillna(1)
    test_df['question_difficulty'] = test_df['content_id'].map(content_agg['sum'] / content_agg['count']).fillna(0.7)
    test_df = test_df.fillna(0)
    #类别特征转换
    for columns in ['user_id','content_id',\
                'task_container_id','prior_question_had_explanation',\
                'bundle_id','part']:
        test_df[columns] = dict_cat_class[columns].transform(test_df[columns])
        print(columns)

    #数值特征归一化
    for columns in ['timestamp','prior_question_elapsed_time','content_count',\
                'question_difficulty','user_correctness']:
 
        test_df[columns] = dict_float_class[columns].transform(test_df[columns])
        print(columns)

#     test_df['tags'] = dict_cat_seq_class['tags'].transform(test_df['tags'])
#     print('tags')
    
    X_test = [np.array(test_df[columns].values.tolist()) for columns in features_columns]
    test_df['answered_correctly'] =  model.predict(X_test, verbose=0, batch_size=512)
    env.predict(test_df.loc[test_df['content_type_id'] == 0, ['row_id', 'answered_correctly']])


user_id
content_id
task_container_id
prior_question_had_explanation
bundle_id
part
timestamp
prior_question_elapsed_time
content_count
question_difficulty
user_correctness
user_id
content_id
task_container_id
prior_question_had_explanation
bundle_id
part
timestamp
prior_question_elapsed_time
content_count
question_difficulty
user_correctness
user_id
content_id
task_container_id
prior_question_had_explanation
bundle_id
part
timestamp
prior_question_elapsed_time
content_count
question_difficulty
user_correctness
user_id
content_id
task_container_id
prior_question_had_explanation
bundle_id
part
timestamp
prior_question_elapsed_time
content_count
question_difficulty
user_correctness
